In [33]:
# We import pytorch and torchvision
import torchvision, torch
import torch.nn as nn
import torch.nn.functional as F
# Import numpy
import numpy as np
import time

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5,), (0.5,))  # Perform normalization on GPU if possible
])

# We load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='data/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='data/', train=False, transform=transform, download=True)

# Charger un dataloader with batch size x
def get_data_loader(dataset, batch_size):
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

def get_batch_format(data_loader):
    images_t, labels_t = next(iter(data_loader))
    print('images.shape:', images_t.shape)
    print('labels.shape:', labels_t.shape)

class Network(nn.Module):
    def __init__(self, input_dim, hidden_dim1, output_dim=10):
        super(Network, self).__init__()
        self.a = nn.Linear(input_dim, hidden_dim1)
        self.b = nn.Linear(hidden_dim1, hidden_dim1)
        self.c = nn.Linear(hidden_dim1, hidden_dim1)
        self.d = nn.Linear(hidden_dim1, output_dim)

    def forward(self, x):
        #x = x.permute(0, 2, 3, 1)
        x = self.a(x)
        x = self.b(x)
        x = self.c(x)
        x = self.d(x)
        return x

# Fonction accuracy pour calculer le total des bonnes réponses
def accuracy(predictions, labels):
    _, predicted_labels = torch.max(predictions, 1)
    correct = (predicted_labels == labels).sum().item()
    return correct / len(labels)

# Charger dans un DataLoader le data set de train CIFAR10
train_loader = get_data_loader(train_dataset, 128) # On charge par batch
# On charge le dataset de testto
test_loader = get_data_loader(test_dataset, 10000) # On charge tout le dataset de test
# Print the format of the batch
get_batch_format(train_loader)

# On recupère le device
deviceGPU = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# On crée une instance de la classe MyClass
modelNet = Network(3*32*32, 32, 10)

# On envoie le modèle sur le device
modelNet = modelNet.to(deviceGPU)

# On crée une fonction de loss
lossFn = F.cross_entropy
learningRate = 0.0001

# On crée un optimiseur
opt = torch.optim.Adam(modelNet.parameters(), lr=learningRate, weight_decay=0.0001)

def off_load_on_gpu(train_loader, test_loader, device):
    train_loader_gpu = [(x.to(device).view(x.shape[0], -1), y.to(device)) for x, y in train_loader]
    test_loader_gpu = [(x.to(device).view(x.shape[0], -1), y.to(device)) for x, y in test_loader]
    return train_loader_gpu, test_loader_gpu

def training_cycle(model, train_loader, device):
    model.train()
    for x, y in train_loader:
        #x = x.view(x.shape[0], -1).to(device, non_blocking=True)
        #y = y.to(device, non_blocking=True)
        preds = model(x)
        loss = lossFn(preds, y)
        # Compute gradients
        loss.backward()
        # Update parameters
        opt.step()
        # Reset gradients to 0
        opt.zero_grad()
    
def validation_cycle(model, test_loader, epoch, num_epochs, device):
    model.eval()
    total_loss = 0
    total_correct = 0
    with torch.no_grad():
        for x, y in test_loader:
            #x = x.view(x.shape[0], -1).to(device, non_blocking=True)
            #y = y.to(device, non_blocking=True)
            preds = model(x)
            loss = lossFn(preds, y)
            total_loss += loss.item()
            total_correct += accuracy(preds, y)

    avg_loss = total_loss / len(test_loader)
    avg_acc = total_correct / len(test_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {avg_acc * 100:.2f}%")

def fit_one_cycle(model, train_loader, test_loader, epoch, num_epochs, device): 
    # Training
    training_cycle(model, train_loader, device)
    
    # Validation
    validation_cycle(model, test_loader, epoch, num_epochs, device)
    
# Training loop
num_epochs = 250
# Time to train
time_to_train = time.time()

# On charge les données sur le GPU
train_loader_gpu, test_loader_gpu = off_load_on_gpu(train_loader, test_loader, deviceGPU)

for epoch in range(num_epochs):
    fit_one_cycle(modelNet, train_loader_gpu, test_loader_gpu, epoch, num_epochs, deviceGPU)

time_to_train = time.time() - time_to_train
print(f"Time to train: {time_to_train:.2f} seconds")

Files already downloaded and verified
Files already downloaded and verified
images.shape: torch.Size([128, 3, 32, 32])
labels.shape: torch.Size([128])
Epoch 1/250, Loss: 1.8406, Accuracy: 36.62%
Epoch 2/250, Loss: 1.7816, Accuracy: 38.60%
Epoch 3/250, Loss: 1.7524, Accuracy: 39.74%
Epoch 4/250, Loss: 1.7362, Accuracy: 40.43%
Epoch 5/250, Loss: 1.7267, Accuracy: 40.70%
Epoch 6/250, Loss: 1.7207, Accuracy: 40.84%
Epoch 7/250, Loss: 1.7168, Accuracy: 40.99%
Epoch 8/250, Loss: 1.7140, Accuracy: 40.91%
Epoch 9/250, Loss: 1.7122, Accuracy: 40.86%
Epoch 10/250, Loss: 1.7108, Accuracy: 40.88%
Epoch 11/250, Loss: 1.7099, Accuracy: 41.02%
Epoch 12/250, Loss: 1.7094, Accuracy: 41.08%
Epoch 13/250, Loss: 1.7090, Accuracy: 41.18%
Epoch 14/250, Loss: 1.7089, Accuracy: 41.28%
Epoch 15/250, Loss: 1.7089, Accuracy: 41.16%
Epoch 16/250, Loss: 1.7090, Accuracy: 41.13%
Epoch 17/250, Loss: 1.7092, Accuracy: 41.16%
Epoch 18/250, Loss: 1.7094, Accuracy: 41.20%
Epoch 19/250, Loss: 1.7097, Accuracy: 41.18%
Epo